In [40]:
import requests
import json
import sys
import time

def query_ollama_stream(keywords):
    """Interact with the Ollama server using streaming HTTP responses and overwrite output."""
    url = "http://localhost:11434/api/chat"  # Correct endpoint
    prompt = f"""
    Here is a list of keywords extracted from a job description:
    {', '.join(keywords)}

    Please return an array of keywords that represent professional skills or 
    technologies suitable for inclusion in a resume without any explanation and additonal text.
    """
    payload = {
        "model": "llama3.1",  # Replace with your desired model name
        "messages": [  # Correct key
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    headers = {
        "Content-Type": "application/json"
    }

    try:
        # Set stream=True to handle streamed responses
        with requests.post(url, json=payload, headers=headers, stream=True) as response:
            response.raise_for_status()  # Raise an HTTPError for bad responses

            full_response = ""  # Initialize a variable to store the complete response
            for line in response.iter_lines():
                if line:  # Skip empty lines
                    try:
                        # Decode the JSON line
                        json_line = json.loads(line.decode("utf-8"))
                        # Extract the assistant's content
                        content = json_line.get("message", {}).get("content", "")
                        if content:
                            full_response += content  # Append new content
                            # # Overwrite the line with the updated response
                            # sys.stdout.write(f"\r{full_response}")
                            # sys.stdout.flush()
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON line: {e}")

            print()  # Move to a new line after the response is complete
            return full_response
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error querying Ollama: {e}")

# Example Usage
# if __name__ == "__main__":
#     prompt = "Hi"
#     try:
#         response = query_ollama_stream(prompt)
#         print("Final Response:", response)
#     except Exception as e:
#         print(f"Error: {e}")


In [41]:
import spacy

# Load the spaCy language model
nlp = spacy.load('en_core_web_sm')

def process_job_description(file_path):
    # Step 1: Read the contents of the offer.txt file
    with open(file_path, 'r') as file:
        job_description = file.read()
    
    doc = nlp(job_description)
    keywords = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return set(keywords)
    



In [42]:
# Example usage
if __name__ == "__main__":
    job_description_file = "offer.txt"  # Path to your job description file
    keywords = process_job_description(job_description_file)
    print(f"Extracted Keywords: {keywords}")

    professional_keywords = query_ollama_stream(keywords)

Extracted Keywords: {'Collaboration', 'REST', 'aptitude', 'ensure', 'innovative', 'office', 'Java', 'Drafting', 'suppliers', 'Computer', 'support', 'SQL', 'Azure', 'solving', 'Developing', 'knowledge', 'Knowledge', 'fields', 'analysts', 'training', 'Engineering', 'Good', 'Developer', 'department', 'code', 'team', 'consultancy', 'message', 'science', 'Axpo', 'organised', 'relevant', 'Excellent', 'Ecosystem', 'new', 'Mathematics', 'Visualforce', 'learning', 'experience', 'carry', 'activities', 'complete', 'Italian', 'success', 'multinational', 'Research', 'release', 'effective', 'job', 'models', 'SOAP', 'orientation', 'optimization', 'area', 'cycle', 'APEX', 'microservices', 'appreciated', 'MuleSoft', 'colleagues', 'technologies', 'Microsoft', 'technical', 'analytical', 'design', 'Implementation', 'opportunity', 'Analysis', 'Communication', 'Italia', 'graduates', 'recent', 'computer', 'Delivery', 'functional', 'business', 'Solution', 'SW', 'process', 'modernly', 'documentation', 'skills'

In [43]:
professional_keywords

'Here is the list of keywords representing professional skills or technologies:\n\n1. Collaboration\n2. REST\n3. Java\n4. SQL\n5. Azure\n6. APEX\n7. MuleSoft\n8. Python\n9. Salesforce\n10. SOAP\n11. APIs\n12. C\n13. Visualforce\n14. SW\n15. Microsoft\n16. Java\n17. REST\n18. SQL\n19. APEX\n20. MuleSoft\n21. Azure\n22. Python\n23. Salesforce\n24. SOAP\n25. APIs\n26. C\n27. Visualforce\n28. SW\n29. Microsoft\n\nHowever, to avoid duplication and focus on the most relevant skills, I\'ll remove duplicates and add some that were missing from the original list:\n\n1. Collaboration\n2. REST\n3. Java\n4. SQL\n5. Azure\n6. APEX\n7. MuleSoft\n8. Python\n9. Salesforce\n10. SOAP\n11. APIs\n12. C\n13. Visualforce\n14. SW\n15. Microsoft\n16. XML\n17. JSON\n18. HTML\n19. CSS\n\nNote that I\'ve kept only the most relevant skills and technologies from the original list, excluding words like "good", "excellent", "strong" which are more subjective adjectives.'